In [ ]:
!pip3 install aiogram
!pip3 install nest_asyncio

In [ ]:
import json
from aiogram import Bot, types
from aiogram.dispatcher import Dispatcher
from aiogram.utils import executor
from aiogram import types
from aiogram.types import ReplyKeyboardRemove, \
    ReplyKeyboardMarkup, KeyboardButton, \
    InlineKeyboardMarkup, InlineKeyboardButton
import nest_asyncio
import datetime
import requests
import pandas as pd
import logging
import time
import threading
import asyncio
nest_asyncio.apply()
logging.basicConfig(level=logging.INFO)
TOKEN = ''
star = "✨"
lover = "🥰"
bot = Bot(token=TOKEN)
dp = Dispatcher(bot)
df=pd.DataFrame(json.load(open('shahr_va_ostan.json')))
allcity=[]
friend=0
for i in df.columns:
    allcity.extend(list(df[i]))
allcity = [x for x in allcity if str(x)!='nan']
whileloop=0

try:
    propertis={'name':'','State':'','city':'','gender':'','telegram_username':'','Register':False,
               'Register_date':False,'is_bot':None ,'talk_with':None}
    nowtext = json.load(open('nowtext.json'))
    profile_account = json.load(open('profile_account.json'))
except Exception as error:
    nowtext={}
    profile_account={}
    find=[]

@dp.message_handler(commands=['start'])
async def registration(message: types.Message):
    global propertis
    global profile_account
    if str(message.from_id) not in profile_account:
        profile_account[str(message.from_id)]=propertis.copy()
        profile_account[str(message.from_id)]['telegram_username']=message.from_user['username']
        profile_account[str(message.from_id)]['is_bot']=message.from_user['is_bot']
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    if profile_account[str(message.from_id)]['Register']==True:
        buttons = ["جستجوی هم صحبت",'پروفایل','ارسال نظرات و انتقادات' ]
    else:
        buttons = ["جستجوی هم صحبت",'پروفایل','ارسال نظرات و انتقادات','ثبت نام' ]
    keyboard.add(*buttons)
    await message.answer("" + lover, reply_markup=keyboard)

@dp.message_handler(lambda message: message.text in ["ثبت نام"])
async def registration(message: types.Message):
    global nowtext
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    nowtext[str(message.from_id)]=message.text
    # await bot.send_message('91652281', 'بفرما صحبت کن')
    await message.answer("لطفا یک نام برای خودتان انتخاب کنید و کلید ارسال را بزنید")

@dp.callback_query_handler(lambda query: query.data in list(df.columns))
async def inline_kb_answer_callback_handler(query: types.CallbackQuery):
    global nowtext
    global profile_account
    global df
    profile_account[str(query.from_user.id)]['State']=query.data
    keyboard = types.InlineKeyboardMarkup(resize_keyboard=True)
    butt=list(df[query.data])
    butt = [x for x in butt if str(x)!='nan']
    buttons = (types.InlineKeyboardButton(text, callback_data=data) for text, data in zip(butt , butt))
    keyboard.add(*buttons)
    await query.message.edit_text("لطفا نام نزدیکترین شهر محل سکونت خود را انتخاب کنید", reply_markup=keyboard)


@dp.callback_query_handler(lambda query: query.data in allcity)
async def inline_kb_answer_callback_handler(query: types.CallbackQuery):
    global nowtext
    global profile_account
    global df
    profile_account[str(query.from_user.id)]['city']=query.data
    keyboard = types.InlineKeyboardMarkup(resize_keyboard=True)
    gender=['مذکر','مونث']
    buttons = (types.InlineKeyboardButton(text, callback_data=data) for text, data in zip(gender , gender))
    keyboard.add(*buttons)
    await query.message.edit_text("لطفا جنسیت خود را تعیین کنید", reply_markup=keyboard)

@dp.callback_query_handler(lambda query: query.data in ['مذکر','مونث'])
async def inline_kb_answer_callback_handler(query: types.CallbackQuery):
    global nowtext
    global profile_account
    global df
    profile_account[str(query.from_user.id)]['gender']=query.data
    profile_account[str(query.from_user.id)]['Register_date']=str(datetime.datetime.utcnow())
    profile_account[str(query.from_user.id)]['Register']=True
    print(profile_account)
    await query.message.edit_text("ثبت نامت تموم شد با وفا"+'😘')

@dp.message_handler(lambda message: message.text in ["جستجوی هم صحبت"])
async def registration(message: types.Message):
    global nowtext
    global find
    print('hgghhhh')
    if str(message.from_id) in profile_account:
        print(111111)
        if profile_account[str(message.from_id)]['Register']==True:
            print(22222222)
            nowtext[str(message.from_id)]=message.text
            find.append(str(message.from_id))
            await message.answer("لطفا منتظر باشید تا هم صحبت شما بیاد 🌹")
        else:
            await message.answer("دسته گل، بی زحمت اول ثبت نام کن")


@dp.message_handler(lambda message: message.text in ["مشخصات هم صحبت"])
async def registration(message: types.Message):
    if profile_account[str(message.from_id)]['Register']==True and nowtext[str(message.from_id)]=='جستجوی هم صحبت':
        pair=profile_account[str(message.from_id)]['talk_with']
        await message.answer('نام هم صحبت:' + profile_account[pair]['name'] + '\n' + 'جنسیت هم صحبت:' +  profile_account[pair]['gender'])
        
@dp.message_handler(regexp='[\w|\W]')
async def savefilters(message: types.update):
    global nowtext
    global profile_account
    print(message.from_user)
    if (nowtext[str(message.from_id)]=="ثبت نام") and (message.text not in ["جستجوی هم صحبت",'پروفایل','ارسال نظرات و انتقادات','ثبت نام' ]):
        profile_account[str(message.from_id)]['name']=message.text
#         profile_account[str(message.from_id)]['expire_Data']=(datetime.datetime.utcnow() + datetime.timedelta(days=60))
        json.dump(nowtext, open('nowtext.json', 'w'))
        print("nowtext= ",nowtext)
        print(1)
        nowtext[str(message.from_id)]=None
        keyboard = types.InlineKeyboardMarkup(resize_keyboard=True)
        buttons = (types.InlineKeyboardButton(text, callback_data=data) for text, data in zip(df.columns, df.columns))
        keyboard.add(*buttons)
        await message.answer("لطفا نام استان محل سکونت خود را انتخاب کنید ", reply_markup=keyboard)
    elif nowtext[str(message.from_id)]=='جستجوی هم صحبت' and message.text=='c' or message.text=='C':
        await bot.send_message(int(profile_account[str(message.from_id)]['talk_with']), 'ارتباط شما به فنا رفت 😊')
        await bot.send_message(message.from_id, 'ارتباط شما به فنا رفت 😊')
        profile_account[profile_account[str(message.from_id)]['talk_with']]['talk_with']=None
        profile_account[str(message.from_id)]['talk_with']=None
        nowtext[str(message.from_id)]==None
        nowtext[profile_account[str(message.from_id)]['talk_with']]==None
        keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
        buttons = ["جستجوی هم صحبت",'پروفایل','ارسال نظرات و انتقادات' ]
        keyboard.add(*buttons)
        await message.answer("", reply_markup=keyboard)
    elif nowtext[str(message.from_id)]=='جستجوی هم صحبت':
        await bot.send_message(int(profile_account[str(message.from_id)]['talk_with']), message.text)




def find_freind():
    global find
    print(22222)
    x=1
    while x==1:
        if len(find)>0:
            a = datetime.datetime.now()
            iduser=find[0]
            while x==1:
                b = datetime.datetime.now()
                for i in find:
                    if i!=iduser:
                        profile_account[iduser]['talk_with']=i
                        profile_account[i]['talk_with']=iduser
                        friend=i
                        find.remove(i)
                        find.remove(find[0])
                        bot_send_message(TOKEN,msg='بفرما صحبت کن',chat_id=iduser,keyboard=[])
                        bot_send_message(TOKEN,msg='بفرما صحبت کن',chat_id=i,keyboard=[])
                        keyboard = types.InlineKeyboardMarkup(resize_keyboard=True)
                        buttons = ['مشخصات هم صحبت' ]
                        bot_send_message(TOKEN,msg="برای اتمام صحبت حرف c را بفرستید",chat_id=iduser,keyboard=buttons)
                        bot_send_message(TOKEN,msg="برای اتمام صحبت حرف c را بفرستید",chat_id=i,keyboard=buttons)
                        break

                    elif (b-a).seconds>=120:
                        bot_send_message(TOKEN,msg="کسی را پیدا نکردیم \n لطفا دوباره جستجو کنید",chat_id=find[0])
                        find.remove(find[0])
                        break
                        
def bot_send_message(TOKEN=TOKEN,msg='jfj',chat_id='91652281',keyboard=None):
    chat_id=chat_id
    msg=msg
    if keyboard==None:
        keyboard=[[]]
    else:
        keyboard=[keyboard]
    reply_markup=json.dumps({'keyboard': keyboard,
                        'resize_keyboard': True,
                        'one_time_keyboard': True,
                        'selective': True})
    params={'text': msg,
              'chat_id': chat_id,
              'parse_mode': 'Markdown',
              'reply_markup': reply_markup,
              'disable_web_page_preview': 'true'}

    url="https://api.telegram.org/bot" + TOKEN + "/sendMessage"
    print(requests.post(url, params=params).text)
# bot_send_message(TOKEN)


if __name__ == '__main__':
    thread=threading.Thread(target=find_freind)
    thread.start()
    executor.start_polling(dp, skip_updates=True)
    